In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import torch
from src import models, data, lens, functional
from src.utils import experiment_utils

import logging
from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

torch.__version__

'1.13.1'

In [3]:
device = "cuda:0"
mt = models.load_model("mamba-3b", device=device, fp16=False)
# mt = models.load_model("gptj", device=device, fp16=True)

2024-01-31 17:36:48 src.models INFO     loading state-spaces/mamba-2.8b-slimpj (device=cuda:0, fp16=False)


2024-01-31 17:36:48 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-01-31 17:36:48 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /state-spaces/mamba-2.8b-slimpj/resolve/main/config.json HTTP/1.1" 200 0
2024-01-31 17:37:06 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /state-spaces/mamba-2.8b-slimpj/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
2024-01-31 17:37:15 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/gpt-neox-20b/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2024-01-31 17:37:15 src.models INFO     dtype: torch.float32, device: cuda:0, memory: 10.31 GB


In [4]:
dataset = data.load_dataset()

# relation_names = [r.name for r in dataset.relations]
# relation_options = Menu(choices = relation_names, value = relation_names)
# show(relation_options) # !caution: tested in a juputer-notebook. baukit visualizations are not supported in vscode.

2024-01-31 17:37:15 src.data DEBUG    no paths provided, using default data dir: /home/local_arnab/Codes/relations/notebooks/../data
2024-01-31 17:37:15 src.data DEBUG    /home/local_arnab/Codes/relations/notebooks/../data is directory, globbing for json files...
2024-01-31 17:37:15 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/characteristic_gender.json
2024-01-31 17:37:15 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/degree_gender.json
2024-01-31 17:37:15 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_birthplace.json
2024-01-31 17:37:15 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_gender.json
2024-01-31 17:37:15 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_religion.json
2024-01-31 17:37:15 src.data DEBUG    found relation 

In [5]:
relation_name = "adjective superlative"
relation = dataset.filter(relation_names=[relation_name])[0]
print(f"{relation.name} -- {len(relation.samples)} samples")
print("------------------------------------------------------")

experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
train, test = relation.split(5)
print("\n".join([sample.__str__() for sample in train.samples]))

2024-01-31 17:37:15 src.data DEBUG    filtering to only relations: ['adjective superlative']
adjective superlative -- 80 samples
------------------------------------------------------
2024-01-31 17:37:15 src.utils.experiment_utils INFO     setting all seeds to 12345
quick -> quickest
rich -> richest
thick -> thickest
strong -> strongest
great -> greatest


In [6]:
################### hparams ###################
layer = 20
beta = 8
###############################################

In [7]:
len(train.samples)

5

In [8]:
hasattr(mt.model, "backbone")

True

In [9]:
models.determine_layer_paths(mt)[layer]

'backbone.layers.20'

In [10]:
prompt = mt.tokenizer.eos_token + " Michael Jordan professionally played the sport of"

tokenized = mt.tokenizer(prompt, return_tensors="pt").to(device)

In [11]:
import baukit

layer_out = models.determine_layer_paths(mt)[layer]
mixer_out = models.determine_layer_paths(mt)[layer] + ".mixer"
layer_in = models.determine_layer_paths(mt)[layer+1]
final_layer = models.determine_layer_paths(mt)[-1]
final_mixer = models.determine_layer_paths(mt)[-1] + ".mixer"

with baukit.TraceDict(
    module=mt.model,
    layers=[
        layer_out,
        mixer_out,
        layer_in,
        final_mixer,
        final_layer,
    ],
    retain_input=True,
) as traces:
    output = mt(**tokenized)

In [12]:
traces[final_mixer].output

tensor([[[ 0.5940,  1.9729, -0.7331,  ...,  2.2618, -2.4105,  0.6027],
         [ 1.2616,  0.6781, -0.8456,  ...,  1.3468, -2.0678,  3.0296],
         [-0.0950,  1.0767, -0.0086,  ..., -0.8217,  0.7617,  0.6352],
         ...,
         [ 7.5944,  2.9077, -2.3740,  ..., -2.0662,  0.9440, -1.2279],
         [-7.4484, -2.3972, -2.6358,  ..., -4.6405, -0.2366,  1.1990],
         [-2.2326, -0.7476, -2.6844,  ..., -3.6317,  1.6523, -0.6629]]],
       device='cuda:0', grad_fn=<MambaInnerFnBackward>)

In [13]:
traces[final_layer].output

(tensor([[[ 0.5940,  1.9729, -0.7331,  ...,  2.2618, -2.4105,  0.6027],
          [ 1.2616,  0.6781, -0.8456,  ...,  1.3468, -2.0678,  3.0296],
          [-0.0950,  1.0767, -0.0086,  ..., -0.8217,  0.7617,  0.6352],
          ...,
          [ 7.5944,  2.9077, -2.3740,  ..., -2.0662,  0.9440, -1.2279],
          [-7.4484, -2.3972, -2.6358,  ..., -4.6405, -0.2366,  1.1990],
          [-2.2326, -0.7476, -2.6844,  ..., -3.6317,  1.6523, -0.6629]]],
        device='cuda:0', grad_fn=<MambaInnerFnBackward>),
 tensor([[[-2.9466,  0.8577,  3.0205,  ...,  0.9754, -0.4319, -1.6834],
          [-0.6205,  0.6147,  0.7412,  ...,  2.2544, -3.0231,  1.3576],
          [-0.1985, -1.1145, -0.9235,  ..., -0.4823, -0.7575, -1.0683],
          ...,
          [ 4.5634,  1.1499, -1.6278,  ..., -5.0547, -6.7761, -7.6065],
          [ 3.3821, -3.7479, -3.7378,  ..., -5.0487, -9.0620, -2.6670],
          [ 3.8406, -3.8000, -4.0094,  ..., -1.7394, -5.0764, -2.5944]]],
        device='cuda:0', grad_fn=<LayerNormF

In [14]:
torch.allclose(traces[mixer_out].output, traces[layer_out].output[0])

True

In [15]:
hasattr(mt.model, "backbone")

True

In [16]:
print("something")

something


In [25]:
from src.operators import JacobianIclMeanEstimator

estimator = JacobianIclMeanEstimator(
    mt = mt, 
    h_layer = layer,
    beta = beta
)
operator = estimator(
    relation.set(
        samples=train.samples, 
    )
)

2024-01-31 17:49:32 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The superlative form of great is greatest
The superlative form of strong is strongest
The superlative form of quick is
2024-01-31 17:49:32 src.operators DEBUG    note that subject=quick, h_index=27
h.shape=torch.Size([2560]) | h.requires_grad=True
============> h.shape=torch.Size([2560])  |  hs[0, _h_index].shape=torch.Size([2560])  |  hs.shape=torch.Size([1, 29, 2560])
True
hs[0, _h_index]=tensor([ 0.0283,  0.0254, -0.0428,  ...,  0.1103, -0.0791, -0.0631],
       device='cuda:0', grad_fn=<SelectBackward0>)
hs=tensor([[[-1.3572e-01, -3.0308e-01,  1.7413e-01,  ...,  2.6154e-01,
          -1.8122e-01,  2.8485e-02],
         [ 2.9972e-02, -1.6609e-01,  2.7416e-01,  ...,  1.4466e-01,
           1.0326e-01, -6.8223e-02],
         [ 3.1619e-01, -1.8137e-02,  1.3349e-01,  ..., -1.3597e-01,
           7.1534e-03,  2.8467e-01],
         ...,
         [ 6.2527e-02, -9.4892e-02,  3.4228e-02,  ...,  1.3772e-01,
    

  0%|          | 0/2560 [00:00<?, ?it/s]

2024-01-31 17:56:10 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The superlative form of great is greatest
The superlative form of quick is quickest
The superlative form of rich is
2024-01-31 17:56:10 src.operators DEBUG    note that subject=rich, h_index=28
h.shape=torch.Size([2560]) | h.requires_grad=True
============> h.shape=torch.Size([2560])  |  hs[0, _h_index].shape=torch.Size([2560])  |  hs.shape=torch.Size([1, 30, 2560])
True
hs[0, _h_index]=tensor([ 0.4376, -0.0241, -0.0391,  ...,  0.1739, -0.0989, -0.0482],
       device='cuda:0', grad_fn=<SelectBackward0>)
hs=tensor([[[-0.1357, -0.3031,  0.1741,  ...,  0.2615, -0.1812,  0.0285],
         [ 0.0300, -0.1661,  0.2742,  ...,  0.1447,  0.1033, -0.0682],
         [ 0.3162, -0.0181,  0.1335,  ..., -0.1360,  0.0072,  0.2847],
         ...,
         [ 0.0635, -0.0819,  0.0294,  ...,  0.1513, -0.0310, -0.0109],
         [ 0.4376, -0.0241, -0.0391,  ...,  0.1739, -0.0989, -0.0482],
         [ 0.0713, -0.0845,  0.0192,

  0%|          | 0/2560 [00:00<?, ?it/s]

2024-01-31 18:03:20 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The superlative form of quick is quickest
The superlative form of strong is strongest
The superlative form of thick is
2024-01-31 18:03:20 src.operators DEBUG    note that subject=thick, h_index=28
h.shape=torch.Size([2560]) | h.requires_grad=True
============> h.shape=torch.Size([2560])  |  hs[0, _h_index].shape=torch.Size([2560])  |  hs.shape=torch.Size([1, 30, 2560])
True
hs[0, _h_index]=tensor([ 0.1792, -0.1925,  0.1866,  ...,  0.1049,  0.0875, -0.1577],
       device='cuda:0', grad_fn=<SelectBackward0>)
hs=tensor([[[-0.1357, -0.3031,  0.1741,  ...,  0.2615, -0.1812,  0.0285],
         [ 0.0300, -0.1661,  0.2742,  ...,  0.1447,  0.1033, -0.0682],
         [ 0.3162, -0.0181,  0.1335,  ..., -0.1360,  0.0072,  0.2847],
         ...,
         [ 0.0618, -0.0891,  0.0310,  ...,  0.1397, -0.0154, -0.0161],
         [ 0.1792, -0.1925,  0.1866,  ...,  0.1049,  0.0875, -0.1577],
         [ 0.0570, -0.0134,  0.0

  0%|          | 0/2560 [00:00<?, ?it/s]

2024-01-31 18:10:20 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The superlative form of thick is thickest
The superlative form of great is greatest
The superlative form of strong is
2024-01-31 18:10:20 src.operators DEBUG    note that subject=strong, h_index=28
h.shape=torch.Size([2560]) | h.requires_grad=True
============> h.shape=torch.Size([2560])  |  hs[0, _h_index].shape=torch.Size([2560])  |  hs.shape=torch.Size([1, 30, 2560])
True
hs[0, _h_index]=tensor([ 0.0335,  0.0232, -0.3097,  ...,  0.1790,  0.0421,  0.0165],
       device='cuda:0', grad_fn=<SelectBackward0>)
hs=tensor([[[-0.1357, -0.3031,  0.1741,  ...,  0.2615, -0.1812,  0.0285],
         [ 0.0300, -0.1661,  0.2742,  ...,  0.1447,  0.1033, -0.0682],
         [ 0.3162, -0.0181,  0.1335,  ..., -0.1360,  0.0072,  0.2847],
         ...,
         [ 0.0646, -0.0785,  0.0285,  ...,  0.1414, -0.0181, -0.0070],
         [ 0.0335,  0.0232, -0.3097,  ...,  0.1790,  0.0421,  0.0165],
         [ 0.0273, -0.0260,  0.0

  0%|          | 0/2560 [00:00<?, ?it/s]

2024-01-31 18:17:21 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The superlative form of strong is strongest
The superlative form of strong is strongest
The superlative form of great is
2024-01-31 18:17:21 src.operators DEBUG    note that subject=great, h_index=27
h.shape=torch.Size([2560]) | h.requires_grad=True
============> h.shape=torch.Size([2560])  |  hs[0, _h_index].shape=torch.Size([2560])  |  hs.shape=torch.Size([1, 29, 2560])
True
hs[0, _h_index]=tensor([ 0.1518,  0.0916,  0.1254,  ...,  0.1513,  0.0455, -0.1086],
       device='cuda:0', grad_fn=<SelectBackward0>)
hs=tensor([[[-0.1357, -0.3031,  0.1741,  ...,  0.2615, -0.1812,  0.0285],
         [ 0.0300, -0.1661,  0.2742,  ...,  0.1447,  0.1033, -0.0682],
         [ 0.3162, -0.0181,  0.1335,  ..., -0.1360,  0.0072,  0.2847],
         ...,
         [ 0.0586, -0.0954,  0.0282,  ...,  0.1394, -0.0248, -0.0157],
         [ 0.1518,  0.0916,  0.1254,  ...,  0.1513,  0.0455, -0.1086],
         [ 0.1030, -0.0233,  0

  0%|          | 0/2560 [00:00<?, ?it/s]

In [19]:
mt.model

MambaLMHeadModel(
  (backbone): MixerModel(
    (embedding): Embedding(50280, 2560)
    (layers): ModuleList(
      (0): Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=2560, out_features=10240, bias=False)
          (conv1d): Conv1d(5120, 5120, kernel_size=(4,), stride=(1,), padding=(3,), groups=5120)
          (act): SiLU()
          (x_proj): Linear(in_features=5120, out_features=192, bias=False)
          (dt_proj): Linear(in_features=160, out_features=5120, bias=True)
          (out_proj): Linear(in_features=5120, out_features=2560, bias=False)
        )
        (norm): RMSNorm()
      )
      (1): Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=2560, out_features=10240, bias=False)
          (conv1d): Conv1d(5120, 5120, kernel_size=(4,), stride=(1,), padding=(3,), groups=5120)
          (act): SiLU()
          (x_proj): Linear(in_features=5120, out_features=192, bias=False)
          (dt_proj): Linear(in_features=160, out_features=512

In [26]:
operator.metadata

{'Jh': [tensor([ 0.7549, -0.1307, -0.0154,  ..., -0.1416, -0.7521, -0.7254]),
  tensor([-0.5954,  0.0357, -0.4120,  ...,  0.6513, -0.7774, -0.2348]),
  tensor([-0.4962,  0.4409, -0.0178,  ..., -0.2932, -1.0436,  0.5390]),
  tensor([-0.1647, -0.5315, -0.3003,  ...,  0.5408,  0.3185, -0.6637]),
  tensor([-0.1539, -0.1978,  0.1350,  ...,  0.4827, -0.8228, -0.3690])],
 '|w|': [254.1029510498047,
  206.6474151611328,
  297.90087890625,
  274.16131591796875,
  310.1672668457031],
 '|b|': [273.321533203125,
  266.7347717285156,
  283.2568664550781,
  309.2726135253906,
  285.7717590332031]}

In [27]:
from src.lens import logit_lens
from src import models

logit_lens(mt = mt, h = operator.metadata["Jh"][0].to(models.determine_device(mt)) + operator.bias)

([(' most', 49.004),
  (' greatest', 46.628),
  (' the', 44.854),
  (' richest', 43.963),
  ('\n', 43.172),
  (' (', 42.98),
  (',', 42.945),
  (' best', 42.763),
  (' biggest', 42.729),
  (' f', 42.31)],
 {})

In [28]:
from src.functional import predict_next_token

predict_next_token(
    mt = mt, 
    # prompt = mt.tokenizer.eos_token + " The capital of {} is".format("France"),
    prompt = mt.tokenizer.eos_token + " The superlative of {} is".format("good"),
)

[[PredictedToken(token=' "', prob=0.09106093645095825),
  PredictedToken(token=' not', prob=0.08697717636823654),
  PredictedToken(token=' good', prob=0.07184084504842758),
  PredictedToken(token=' the', prob=0.06490873545408249),
  PredictedToken(token=' better', prob=0.06266643106937408)]]

# Checking $faithfulness$

In [29]:
test = functional.filter_relation_samples_based_on_provided_fewshots(
    mt=mt, test_relation=test, prompt_template=operator.prompt_template, batch_size=4
)

2024-01-31 18:25:48 src.functional DEBUG    filtering for knowns using prompt "<|endoftext|> The superlative form of quick is quickest
The superlative form of rich is richest
The superlative form of thick is thickest
The superlative form of strong is strongest
The superlative form of great is greatest
The superlative form of {} is"


2024-01-31 18:25:51 src.functional DEBUG    sample.subject='shy', sample.object='shyest', predicted=' shy' (p=0.983), known=(✓)
2024-01-31 18:25:51 src.functional DEBUG    sample.subject='tall', sample.object='tallest', predicted=' tall' (p=0.999), known=(✓)
2024-01-31 18:25:51 src.functional DEBUG    sample.subject='healthy', sample.object='healthiest', predicted=' health' (p=0.971), known=(✓)
2024-01-31 18:25:51 src.functional DEBUG    sample.subject='loud', sample.object='loudest', predicted=' loud' (p=0.998), known=(✓)
2024-01-31 18:25:51 src.functional DEBUG    sample.subject='hard', sample.object='hardest', predicted=' hardest' (p=1.000), known=(✓)
2024-01-31 18:25:51 src.functional DEBUG    sample.subject='polite', sample.object='politest', predicted=' polit' (p=0.893), known=(✓)
2024-01-31 18:25:51 src.functional DEBUG    sample.subject='hot', sample.object='hottest', predicted=' hottest' (p=0.995), known=(✓)
2024-01-31 18:25:51 src.functional DEBUG    sample.subject='smooth', 

In [30]:
sample = [s for s in test.samples if s.subject == "happy"][0]
print(sample)
operator(subject = sample.subject).predictions

happy -> happiest
2024-01-31 18:25:54 src.operators DEBUG    computing h from prompt "<|endoftext|> The superlative form of quick is quickest
The superlative form of rich is richest
The superlative form of thick is thickest
The superlative form of strong is strongest
The superlative form of great is greatest
The superlative form of happy is"


[PredictedToken(token=' happ', prob=0.9729710221290588),
 PredictedToken(token='happ', prob=0.014340016059577465),
 PredictedToken(token=' Happ', prob=0.011374825611710548),
 PredictedToken(token=' happiness', prob=0.00113575195427984),
 PredictedToken(token=' happier', prob=9.16872886591591e-05)]

In [31]:
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject],
    h_layer= operator.h_layer,
)
h = hs_and_zs.h_by_subj[sample.subject]

## Approximating LM computation $F$ as an affine transformation

### $$ F(\mathbf{s}, c_r) \approx \beta \, W_r \mathbf{s} + b_r $$

In [32]:
z = 5 * (operator.weight @ h) + operator.bias

lens.logit_lens(
    mt = mt,
    h = z,
    get_proba = True
)

([(' happ', 0.995),
  (' Happ', 0.003),
  ('happ', 0.002),
  (' happiness', 0.0),
  (' happier', 0.0),
  (' happy', 0.0),
  (' richest', 0.0),
  (' unh', 0.0),
  (' Happy', 0.0),
  (' health', 0.0)],
 {})

In [33]:
correct = 0
wrong = 0
for sample in test.samples:
    predictions = operator(subject = sample.subject).predictions
    known_flag = functional.is_nontrivial_prefix(
        prediction=predictions[0].token, target=sample.object
    )
    print(f"{sample.subject=}, {sample.object=}, ", end="")
    print(f'predicted="{functional.format_whitespace(predictions[0].token)}", (p={predictions[0].prob}), known=({functional.get_tick_marker(known_flag)})')
    
    correct += known_flag
    wrong += not known_flag
    
faithfulness = correct/(correct + wrong)

print("------------------------------------------------------------")
print(f"Faithfulness (@1) = {faithfulness}")
print("------------------------------------------------------------")

2024-01-31 18:26:02 src.operators DEBUG    computing h from prompt "<|endoftext|> The superlative form of quick is quickest
The superlative form of rich is richest
The superlative form of thick is thickest
The superlative form of strong is strongest
The superlative form of great is greatest
The superlative form of angry is"


sample.subject='angry', sample.object='angriest', predicted=" ang", (p=0.775745689868927), known=(✓)
2024-01-31 18:26:02 src.operators DEBUG    computing h from prompt "<|endoftext|> The superlative form of quick is quickest
The superlative form of rich is richest
The superlative form of thick is thickest
The superlative form of strong is strongest
The superlative form of great is greatest
The superlative form of bad is"
sample.subject='bad', sample.object='worst', predicted=" bad", (p=0.9749193787574768), known=(✗)
2024-01-31 18:26:02 src.operators DEBUG    computing h from prompt "<|endoftext|> The superlative form of quick is quickest
The superlative form of rich is richest
The superlative form of thick is thickest
The superlative form of strong is strongest
The superlative form of great is greatest
The superlative form of big is"
sample.subject='big', sample.object='biggest', predicted=" biggest", (p=0.5625869035720825), known=(✓)
2024-01-31 18:26:02 src.operators DEBUG    computin

# $causality$

In [34]:
################### hparams ###################
rank = 100
###############################################

In [35]:
experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
test_targets = functional.random_edit_targets(test.samples)

2024-01-31 18:26:10 src.utils.experiment_utils INFO     setting all seeds to 12345


## setup

In [36]:
source = test.samples[0]
target = test_targets[source]

f"Changing the mapping ({source}) to ({source.subject} -> {target.object})"

'Changing the mapping (angry -> angriest) to (angry -> smallest)'

### Calculate $\Delta \mathbf{s}$ such that $\mathbf{s} + \Delta \mathbf{s} \approx \mathbf{s}'$

<p align="center">
    <img align="center" src="../causality_crop" style="width:80%;"/>
</p>

Under the relation $r =\, $*plays the instrument*, and given the subject $s =\, $*Miles Davis*, the model will predict $o =\, $*trumpet* **(a)**; and given the subject $s' =\, $*Cat Stevens*, the model will now predict $o' =\, $*guiter* **(b)**. 

If the computation from $\mathbf{s}$ to $\mathbf{o}$ is well-approximated by $operator$ parameterized by $W_r$ and $b_r$ **(c)**, then $\Delta{\mathbf{s}}$ **(d)** should tell us the direction of change from $\mathbf{s}$ to $\mathbf{s}'$. Thus, $\tilde{\mathbf{s}}=\mathbf{s}+\Delta\mathbf{s}$ would be an approximation of $\mathbf{s}'$ and patching $\tilde{\mathbf{s}}$ in place of $\mathbf{s}$ should change the prediction to $o'$ = *guitar* 

In [37]:
def get_delta_s(
    operator, 
    source_subject, 
    target_subject,
    rank = 100,
    fix_latent_norm = None, # if set, will fix the norms of z_source and z_target
):
    w_p_inv = functional.low_rank_pinv(
        matrix = operator.weight,
        rank=rank,
    )
    hs_and_zs = functional.compute_hs_and_zs(
        mt = mt,
        prompt_template = operator.prompt_template,
        subjects = [source_subject, target_subject],
        h_layer= operator.h_layer,
        z_layer=-1,
    )

    z_source = hs_and_zs.z_by_subj[source_subject]
    z_target = hs_and_zs.z_by_subj[target_subject]
    
    z_source *= fix_latent_norm / z_source.norm() if fix_latent_norm is not None else 1.0
    z_target *= z_source.norm() / z_target.norm() if fix_latent_norm is not None else 1.0

    delta_s = w_p_inv @  (z_target.squeeze() - z_source.squeeze())

    return delta_s, hs_and_zs

delta_s, hs_and_zs = get_delta_s(
    operator = operator,
    source_subject = source.subject,
    target_subject = target.subject,
    rank = rank
)

In [38]:
import baukit

def get_intervention(h, int_layer, subj_idx):
    def edit_output(output, layer):
        if(layer != int_layer):
            return output
        functional.untuple(output)[:, subj_idx] = h 
        return output
    return edit_output

prompt = operator.prompt_template.format(source.subject)

h_index, inputs = functional.find_subject_token_index(
    mt=mt,
    prompt=prompt,
    subject=source.subject,
)

h_layer, z_layer = models.determine_layer_paths(model = mt, layers = [layer, -1])

with baukit.TraceDict(
    mt.model, layers = [h_layer, z_layer],
    edit_output=get_intervention(
#         h = hs_and_zs.h_by_subj[source.subject],         # let the computation proceed as usual
        h = hs_and_zs.h_by_subj[source.subject] + delta_s, # replace s with s + delta_s
        int_layer = h_layer, 
        subj_idx = h_index
    )
) as traces:
    outputs = mt(
        input_ids = inputs.input_ids,
        attention_mask = inputs.attention_mask,
    )

logits = outputs.logits[0][-1] if hasattr(outputs, "logits") else outputs[0][-1]
lens.interpret_logits(
    mt = mt, 
    logits = logits, 
    get_proba=True
)

[(' most', 0.62),
 (' smallest', 0.301),
 (' m', 0.03),
 (' shortest', 0.01),
 (' least', 0.004),
 (' biggest', 0.002),
 (' the', 0.002),
 (' mild', 0.002),
 (' might', 0.002),
 (' me', 0.001)]

## Measuring causality

In [39]:
from src.editors import LowRankPInvEditor

svd = torch.svd(operator.weight.float())
editor = LowRankPInvEditor(
    lre=operator,
    rank=rank,
    svd=svd,
)

In [41]:
# precomputing latents to speed things up
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject for sample in test.samples],
    h_layer= operator.h_layer,
    z_layer=-1,
    batch_size = 2
)

success = 0
fails = 0

for sample in test.samples:
    target = test_targets.get(sample)
    assert target is not None
    edit_result = editor(
        subject = sample.subject,
        target = target.subject
    )
    
    success_flag = functional.is_nontrivial_prefix(
        prediction=edit_result.predicted_tokens[0].token, target=target.object
    )
    
    print(f"Mapping {sample.subject} -> {target.object} | edit result={edit_result.predicted_tokens[0]} | success=({functional.get_tick_marker(success_flag)})")
    
    success += success_flag
    fails += not success_flag
    
causality = success / (success + fails)

print("------------------------------------------------------------")
print(f"Causality (@1) = {causality}")
print("------------------------------------------------------------")

Mapping angry -> smallest | edit result=' smallest' (p=0.917) | success=(✓)
Mapping bad -> biggest | edit result=' biggest' (p=0.987) | success=(✓)
Mapping big -> meanest | edit result=' most' (p=0.857) | success=(✗)
Mapping brave -> saddest | edit result=' sadd' (p=0.957) | success=(✓)
Mapping bright -> hardest | edit result=' hardest' (p=0.999) | success=(✓)
Mapping calm -> longest | edit result=' longest' (p=0.989) | success=(✓)
Mapping cheap -> smoothest | edit result=' smoot' (p=0.675) | success=(✓)
Mapping clean -> fullest | edit result=' full' (p=0.972) | success=(✓)
Mapping cold -> saddest | edit result=' sadd' (p=0.947) | success=(✓)
Mapping crazy -> easiest | edit result=' easiest' (p=0.910) | success=(✓)
Mapping cruel -> smoothest | edit result=' smoot' (p=0.811) | success=(✓)
Mapping dark -> lightest | edit result=' light' (p=0.922) | success=(✓)
Mapping deep -> zestiest | edit result=' most' (p=0.378) | success=(✗)
Mapping dirty -> gentlest | edit result=' most' (p=0.517) 